In [18]:
import os
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D

In [2]:
# check if GPU is available
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


2023-04-25 23:41:30.181156: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.


In [39]:
# Load data
data_path = "../data/windows/"
windows = os.listdir(data_path)

ictal_windows = [w for w in windows if w.split("_")[1].startswith("ictal")]
interictal_windows = [w for w in windows if w.split("_")[1].startswith("interictal")]
preictal_windows = [w for w in windows if w.split("_")[1].startswith("preictal")]
prepreictal_windows = [w for w in windows if w.split("_")[1].startswith("prepreictal")]

len(ictal_windows), len(interictal_windows), len(preictal_windows), len(prepreictal_windows)

(24, 23, 24, 24)

In [40]:
ictal_windows = [pd.read_csv(data_path + w).drop("edf_file", axis=1) 
                 for w in ictal_windows]

interictal_windows = [pd.read_csv(data_path + w).drop("edf_file", axis=1)
                        for w in interictal_windows]

preictal_windows = [pd.read_csv(data_path + w).drop("edf_file", axis=1)
                        for w in preictal_windows]

prepreictal_windows = [pd.read_csv(data_path + w).drop("edf_file", axis=1)
                        for w in prepreictal_windows]

: 

: 

In [ ]:
# split ictal in subwindows of 60*256
ictal_windows = [w.iloc[i:i+60*256] for w in ictal_windows for i in range(0, len(w), 60*256)]

# split interictal in subwindows of 60*256*5
interictal_windows = [w.iloc[i:i+60*256*5] for w in interictal_windows for i in range(0, len(w), 60*256*5)]

# split preictal in subwindows of 60*256*5
preictal_windows = [w.iloc[i:i+60*256*5] for w in preictal_windows for i in range(0, len(w), 60*256*5)]

# split prepreictal in subwindows of 60*256*5
prepreictal_windows = [w.iloc[i:i+60*256*5] for w in prepreictal_windows for i in range(0, len(w), 60*256*5)]

In [ ]:
# add target column
for w in interictal_windows:
    w["target"] = 0
    
for w in ictal_windows:
    w["target"] = 1

for w in preictal_windows:
    w["target"] = 2

for w in prepreictal_windows:
    w["target"] = 3

In [ ]:
# concatenate all windows
all_windows = pd.concat(ictal_windows + interictal_windows + preictal_windows + prepreictal_windows)

# shuffle
all_windows = all_windows.sample(frac=1)

# split in train and test
train = all_windows.iloc[:int(len(all_windows)*0.8)]
test = all_windows.iloc[int(len(all_windows)*0.8):]

# split in X and y
X_train = train.drop("target", axis=1)
y_train = train["target"]

X_test = test.drop("target", axis=1)
y_test = test["target"]

# reshape X
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# build model
model = Sequential()

model.add(Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
# make model
model = Sequential()

model.add(Conv1D(256, 3, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.1))

model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])